In [1]:
import rPPG_Toolbox.dataset

In [5]:
import os


os.getcwd()

'e:\\FirstProj\\rPPG'

In [2]:
dir(rPPG_Toolbox.dataset)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'data_loader']

In [3]:
parser = rPPG_Toolbox.dataset.data_loader.BaseLoader

In [4]:
import numpy as np
from scipy.stats import gaussian_kde

In [14]:
np.matrix(xy)

matrix([[109.4941901 ,  88.8418154 ],
        [110.06596332,  92.24078273]])

In [19]:
xy = np.array([[109.4941901 ,  88.8418154 ],
       [110.06596332,  92.24078273]]).astype(np.float64)
#xy = np.matrix(xy)
gaussian_kde(xy)(xy)

array([52103.26720011, 52103.26720011])

In [21]:
np.linalg.eigh(np.cov(xy))[0]

array([1.42108547e-14, 3.72128822e+02])

In [23]:
np.eye(np.cov(xy))

TypeError: only integer scalar arrays can be converted to a scalar index